# Audio Similarity Search
In this example we will be going over the code required to perform audio similarity searches. This example uses a the PANNs model to extract audio features that are then used with Milvus to build a system that can perform the searches.

A deployable version of a reverse audio search can be found in this directory.

## Data

This example uses the TUT Acoustic scenes 2017 Evaluation dataset, which contains 1622 10-second audio clips that fall within 15 categories: Bus, Cafe,
Car, City center, Forest path, Grocery store,  Home, Lakeside beach, Library, Metro station, Office, Residential area, Train, Tram, and Urban park.

Dataset size: ~ 4.29 GB.


Directory Structure:  
The file loader used in this example requires that all the data be in .wav format due to librosa limitations. The way that files are read also limits the structure to a folder with all the data points. 

## Requirements

|  Packages   |  Servers    |
|-                  | -                 |   
| pymilvus_orm      | milvus-2.0        |
| redis             | redis             |
| librosa           |
| ipython           |
| numpy             |
| panns_inference   |

We have included a requirements-v2.txt file in order to easily satisfy the required packages. 

In [ ]:
%pip install -r requirements-v2.txt

### Starting Milvus Server

This demo uses Milvus 2.0, please refer to the [Install Milvus](https://milvus.io/docs/v2.0.0/install_standalone-docker.md) guide to learn how to use this docker-compose setup. For this example we wont be mapping any local volumes. 

### Starting Redis Server

We are using Redis as a metadata storage service for this example. Code can easily be modified to use a python dictionary, but that usually does not work in any use case outside of quick examples. We need a metadata storage service in order to be able to be able to map between embeddings and their corresponding audio clips.

In [ ]:
! docker run --name redis -d -p 6379:6379 redis

## Code Overview

### Connecting to Servers
We first start off by connecting to the servers. In this case the docker containers are running on localhost and the ports are the default ports. 

In [ ]:
#Connectings to Milvus and Redis
import redis
from pymilvus_orm import connections
from pymilvus_orm.types import DataType
from pymilvus_orm.schema import FieldSchema, CollectionSchema
from pymilvus_orm.collection import Collection
from pymilvus_orm import utility

connections.connect(host = '127.0.0.1', port = 19530)
red = redis.Redis(host = '127.0.0.1', port=6379, db=0)

### Building Collection and Setting Index

The next step involves creating a collection. A collection in Milvus is similar to a table in a relational database, and is used for storing all the vectors. To create a collection, we first must select a name, the dimension of the vectors being stored within, the index_file_size, and metric_type. The index_file_size corresponds to how large each data segmet will be within the collection. More information on this can be found here. The metric_type is the distance formula being used to calculate similarity. In this example we are using the Euclidean distance. 

In [ ]:
#Creating collection

import time

red.flushdb()
time.sleep(.1)
collection_name = "audio_collection"
collection = Collection(name=collection_name)
collection.drop()

#if not utility.has_collection(collection_name):
field1 = FieldSchema(name="id", dtype=DataType.INT64, descrition="int64", is_primary=True,auto_id=True)
field2 = FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, descrition="float vector", dim=2048, is_primary=False)
schema = CollectionSchema(fields=[ field1,field2], description="collection description")
collection = Collection(name=collection_name, schema=schema)   

After creating the collection we want to assign it an index type. This can be done before or after inserting the data. When done before, indexes will be made as data comes in and fills the data segments. In this example we are using IVF_SQ8 which requires the 'nlist' parameter. Each index types carries its own parameters. More info about this param can be found [here](https://milvus.io/docs/v1.0.0/index.md#CPU).

In [ ]:
#Indexing collection

if utility.has_collection(collection_name):
    collection = Collection(name = collection_name)
default_index = {"index_type": "IVF_SQ8", "metric_type": "L2", "params": {"nlist": 16384}}
status = collection.create_index(field_name = "embedding", index_params = default_index)
if not status.code:
    print("Successfully create index in collection:{} with param:{}".format(collection_name, default_index))

### Processing and Storing Audio Files
In order to store the audio tracks in Milvus, we must first get the embeddings. To do this, we start by loading the audio file using Librosa. Once we have the audio clip loaded we can pass it to the PANN model. In this case we are using the panns_inference library to simplfy the importing and processing. Once we recieve the embedding we can push it into Milvus and store each uniqueID and filepath combo into redis. We do this so that we can later access the audio file when displaying the results. 

In [ ]:
import os
import librosa
import numpy as np
from panns_inference import SoundEventDetection, labels, AudioTagging
from diskcache import Cache

data_dir = './TUT-acoustic-scenes-2017-evaluation/audio'
at = AudioTagging(checkpoint_path=None, device='cpu')


def embed_and_save(path, at):
    audio, _ = librosa.core.load(path, sr=32000, mono=True)
    audio = audio[None, :]
    try:
        _, embedding = at.inference(audio)
        embedding = embedding/np.linalg.norm(embedding)
        embedding = embedding.tolist()[0]
        mr = collection.insert([[embedding]])
        ids = mr.primary_keys
        collection.load()
        red.set(str(ids[0]), path)
    except Exception as e:
        print("failed: " + path + "; error {}".format(e))

print("Starting Insert")
for subdir, dirs, files in os.walk(data_dir):
    for file in files:
        path = os.path.join(subdir, file)
        embed_and_save(path, at)
print("Insert Done")
        
        

### Searching
In this example we perform a search on a few randomly selected audio clips. In order to perform the search we must first apply the same processing that was done on the original audio clips. This will result in us having a set of embeddings.

In [ ]:
def get_embed(paths, at):
    embedding_list = []
    for x in paths:
        audio, _ = librosa.core.load(x, sr=32000, mono=True)
        audio = audio[None, :]
        try:
            _, embedding = at.inference(audio)
            embedding = embedding/np.linalg.norm(embedding)
            embedding_list.append(embedding)
        except:
            print("Embedding Failed: " + x)
    return np.array(embedding_list, dtype=np.float32).squeeze()

random_ids = [int(red.randomkey()) for x in range(2)]
search_clips = [x.decode("utf-8") for x in red.mget(random_ids)]
embeddings = get_embed(search_clips, at)
print(embeddings.shape)

We can then take these embeddings and perform a search. The search requires a few arguments: the name of the collection, the vectors being searched for, how many closest vectors to be returned, and the parameters for the index, in this case nprobe. Once performed this example will return the searched clip and the result clips. 

In [ ]:
import IPython.display as ipd

def show_results(query, results, distances):
    print("Query: ")
    ipd.display(ipd.Audio(query))
    print("Results: ")
    for x in range(len(results)):
        print("Distance: " + str(distances[x]))
        ipd.display(ipd.Audio(results[x]))
    print("-"*50)

print(embeddings.shape)

search_params = {"metric_type": "L2", "params": {"nprobe": 16}}


try:
    start = time.time()
    results = collection.search(embeddings, anns_field="embedding", param=search_params, limit=3)
    end = time.time() - start
    print("Search took a total of: ", end)
    for x in range(len(results)):
        query_file = search_clips[x]
        result_files = [red.get(y.id).decode('utf-8') for y in results[x]]
        distances = [y.distance for y in results[x]]
        show_results(query_file, result_files, distances)
except Exception as e:
    print("Failed to search vectors in Milvus: {}".format(e))

## Conclusion
This notebook shows how to search for similar audio clips. 

Check out our [demo system](https://zilliz.com/milvus-demos) to try out different solutions. 